# RAY

In [ ]:
import ray
import pandas as pd

ray.init(include_dashboard=True)

### Управление объектами в Object Store

In [ ]:
y = "Hello world"
y_ref = ray.put(y)

print(y_ref)
print(ray.get(y_ref))

### Tasks

In [ ]:
@ray.remote(num_cpus=1)
def func(a: int) -> int:
    return a + 1

# аргумент автоматически добавляется в хранилище
res_ref = func.remote(2)
print(ray.get(res_ref))

# можно добавить вручную
y = 1
y_ref = ray.put(y)
res_ref = func.remote(y_ref)
print(ray.get(res_ref))

### Actors

In [ ]:
# если нужен объект, который существует в единственном экземпляре
@ray.remote
class Actor:
    def __init__(self):
        self.d = dict()    

    def get(self, k: int) -> str:
        return self.d[k]

    def set(self, k: int, v: str):        
        self.d[k] = v


actor_ref = Actor.remote()
actor_ref.set.remote(2, "Hello")

res_ref = actor_ref.get.remote(2)
print(ray.get(res_ref))

### Datasets

In [ ]:
ds = ray.data.range(10000)
ds = ray.data.read_csv("../data/currencies.csv")
ds.show(5)

ds = ds.repartition(2)

ds_new = ds.map(
    lambda record: {"USD" : 2 * record["USD"] }
)
ds_new.show(5)

def _mapper(df: pd.DataFrame) -> pd.DataFrame:
    df["USD"] *= 2
    return df

ds_new = ds.map_batches(
    _mapper, 
    batch_format="pandas"
)
ds_new.show(5)